In [104]:
import pandas as pd

In [105]:
from sqlalchemy import create_engine
engine = create_engine("postgresql://aaz.chicagobooth.edu/postgres")

Take a sample from `speaker_data_merged`, which has both old (`.y`) and new (`.x`) versions of speaker data.

In [106]:
df = pd.read_sql_query("SELECT * FROM speaker_data_merged_zjy LIMIT 100000", con=engine)

In [107]:
import difflib
from difflib import ndiff, unified_diff

It might be necessary to run `nltk.download()` before doing this part.

In [108]:
from nltk import sent_tokenize

The following code goes through and compares `speaker_text.y` and `speaker_text.x` in each row and prints a Git-like diff output (subtractions marked with `-` and additions marked with `+` and surrounding "lines" for context).

In [132]:
# check whether the difference between strings is due to whitespace only
def check_whitespace(str1, str2):
    str1 = str1[1:]
    str2 = str2[1:]
    return str1.split() == str2.split()

cnt_body = 0
cnt_ws = 0
cnt_other = 0
tmpstr_1 = ''
tmpstr_2 = ''

for i in range(len(df)):
    sents_x = sent_tokenize(df["speaker_text.y"][i].strip())
    sents_y = sent_tokenize(df["speaker_text.x"][i].strip())
    #print(sents_x)
    
    for diff in unified_diff(sents_x, sents_y):
        if diff and type(diff) == unicode:
                    diff = diff.strip()
                    if diff.startswith("+") or diff.startswith("-"):
                        # check for ']]></Body>'
                        if ']]></Body>' in diff:
                            cnt_body += 1
                        # check for white-space / other
                        else:
                            if tmpstr_1 == '':
                                tmpstr_1 = diff
                            elif tmpstr_2 == '':
                                tmpstr_2 = diff
                                if check_whitespace(tmpstr_1, tmpstr_2):
                                    cnt_ws += 1
                                else:
                                    print 'type3, i=',i
                                    cnt_other += 1
                                # clear tmpstr
                                tmpstr_1 = ''
                                tmpstr_2 = ''
                            else:
                                print 'Error: tmp strings are full.'
                                quit()
print 'cnt_body =',cnt_body
print 'cnt_ws =', cnt_ws
print 'cnt_other =',cnt_other

type3, i= 465
type3, i= 503
type3, i= 718
type3, i= 1528
type3, i= 1610
type3, i= 2486
type3, i= 2677
type3, i= 2809
type3, i= 2846
type3, i= 5001
type3, i= 5022
type3, i= 5025
type3, i= 5157
type3, i= 5272
type3, i= 5308
type3, i= 5313
type3, i= 5370
type3, i= 5834
type3, i= 6165
type3, i= 6165
type3, i= 6165
type3, i= 6167
type3, i= 6169
type3, i= 6170
type3, i= 6173
type3, i= 6183
type3, i= 6183
type3, i= 6188
type3, i= 6189
type3, i= 6204
type3, i= 6206
type3, i= 6212
type3, i= 6212
type3, i= 6216
type3, i= 6218
type3, i= 6220
type3, i= 6225
type3, i= 6228
type3, i= 6236
type3, i= 6313
type3, i= 6529
type3, i= 6533
type3, i= 6583
type3, i= 6784
type3, i= 7159
type3, i= 7162
type3, i= 7374
type3, i= 7840
type3, i= 8421
type3, i= 8578
type3, i= 9999
type3, i= 10110
type3, i= 13036
type3, i= 13036
type3, i= 21189
type3, i= 21277
type3, i= 21496
type3, i= 22838
type3, i= 22916
type3, i= 22916
type3, i= 22916
type3, i= 22916
type3, i= 22916
type3, i= 22916
type3, i= 22916
type3, i= 2303

In [145]:
sents_x = sent_tokenize(df["speaker_text.y"][465].strip())
sents_y = sent_tokenize(df["speaker_text.x"][465].strip())
for diff in unified_diff(sents_x, sents_y):
    if diff.startswith("+"):
        tmp1 = diff
    elif diff.startswith("-"):
        tmp2 = diff
tmp1 =tmp1.strip()
tmp2=tmp2.strip()
print tmp1
print tmp2
tmp11=tmp1[1:]
tmp22=tmp2[1:]
print tmp11
print tmp22

+(VIDEO IN PROGRESS)
 If you were pet parents hopefully you found that pretty exciting' it's even more exciting when you go in the store.
-(VIDEO IN PROGRESS) If you were pet parents hopefully you found that pretty exciting' it's even more exciting when you go in the store.
(VIDEO IN PROGRESS)
 If you were pet parents hopefully you found that pretty exciting' it's even more exciting when you go in the store.
(VIDEO IN PROGRESS) If you were pet parents hopefully you found that pretty exciting' it's even more exciting when you go in the store.


In [131]:
# test
sents_x = sent_tokenize(df["speaker_text.y"][334].strip())
sents_y = sent_tokenize(df["speaker_text.x"][334].strip())

    
for diff in unified_diff(sents_x, sents_y):
    if diff and type(diff) == unicode:
        diff = diff.strip()
       
        if diff.startswith("+") or diff.startswith("-"):
            # check for ']]></Body>'
            if ']]></Body>' in diff:
                cnt_body += 1

                print 'yes'
            # check for white-space / other
            else:
                print 'no'

yes


In [118]:
a=sent_tokenize(df["speaker_text.x"][534].strip())  #714, 334

In [80]:
b=sent_tokenize(df["speaker_text.y"][714].strip())

In [86]:
for i in unified_diff(a, b):
    if type(i) == unicode:
        i = i.strip()
        if i.startswith("+"):
            aa = i
        elif i.startswith("-"):
            bb = i
          

In [152]:
print tmp1
print tmp2
check_whitespace(tmp1, tmp2)

+(VIDEO IN PROGRESS)
 If you were pet parents hopefully you found that pretty exciting' it's even more exciting when you go in the store.
-(VIDEO IN PROGRESS) If you were pet parents hopefully you found that pretty exciting' it's even more exciting when you go in the store.


True